<p><center> <img src="doc/header.png" width="1000"/> </p></center>

# LISFLOOD - a distributed hydrological rainfall-runoff model

LISFLOOD is a spatially distributed, semi-physical hydrological rainfall-runoff model that has been developed by the Joint Research Centre (JRC) of the European Commission in late 90ies. Since then LISFLOOD has been applied to a wide range of applications such as all kind of water resourses assessments looking at e.g. the effects of climate and land-use change as well as river regulation measures. Its most prominent application is probably within the [European Flood Awareness System](https://www.efas.eu/) (EFAS) operated under [Copernicus Emergency Management Service](https://emergency.copernicus.eu/) (EMS).

It’s wide applicability is due to its modular structure as well as its temporal and spatial flexibility. The model can be extended with additional modules when need arises, to satisfy the new target objective. In that sense it can be extended to include anything from a better representation of a particular hydrological flow to the implementation of anthropogenic-influenced processes. At the same time the model has been designed to be applied across a wide range of spatial and temporal scales. LISFLOOD is grid-based, and applications so far have employed grid cells of as little as 100 metres for medium-sized catchments, to 5000 metres for modelling the whole of Europe and up to 0.1° (around 10 km) for modelling globally. Long-term water balance can be simulated (using a daily time step), as well as individual flood events (using hourly time intervals, or even smaller).

Although LISFLOOD’s primary output product is channel discharge, all internal rate and state variables (soil moisture, for example) can be written as output as well. All output can be written as grids, or time series at user-defined points or areas. The user has complete control over how output is written, thus minimising any waste of disk space or CPU time.

## Processing

In [ ]:
# Click to EXPAND

#####################################################
###################### IMPORTS ######################
#####################################################

%matplotlib ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

#import mpl_interactions.ipyplot as iplt

import ipyleaflet
import cartopy
import ipywidgets
import xml.etree.ElementTree as ET
import ipywidgets
import datetime
import pandas as pd
import altair as alt
import functools                              # higher-order functions and operations
from ipyfilechooser import FileChooser
import netCDF4
import ipywidgets
import matplotlib.pylab as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
#from carto import scale_bar
from matplotlib import rc
rc('animation', html='jshtml')
from math import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
import glob
import os
import netCDF4
import cartopy.io.img_tiles as cimgt
from matplotlib.colors import ListedColormap




####################################################
############### FUNCTION DEFINITIONS ###############
####################################################

# PRELIMINARY
def addData(df, path, setting):
    df_temp = pd.read_csv('%s/out/dis_run.tss' % fc.selected_path, delim_whitespace=True, header=None)
    df_temp = df_temp.drop([0,1,2,3], 0)
    df_temp = df_temp.drop([2,3,4,5,6,7,8,9], 1)
    df_temp.columns = ['date', 'value']
    df_temp.insert(0, 'setting', [setting] * len(df_temp.index), True)

    return pd.concat([df, df_temp], axis=0, ignore_index=True)

# callback function to show settings after XML files are chosen
def show_settings(chooser):
    
    # checks whether PRE-RUN and RUN settings files are given
    if fc[0].selected != None and fc[1].selected != None:

        global tree
        global StepStart
        global StepStart_picker
        global StepEnd
        global StepEnd_picker
        global calibration_parameters_xml
        global calibration_sliders
        global optional_modules_xml
        global module_checkboxes
        global m

        # opens settings file of PRE-RUN ([0]) and RUN ([1]) in list
        tree = [ET.parse(fc[0].selected), ET.parse(fc[1].selected)]
        # gets root of XML
        root = [tree[0].getroot(), tree[1].getroot()]

        # finds simulation start date in XML files
        StepStart = [root[0].find("./lfuser/group/textvar[@name='StepStart']"),
                     root[1].find("./lfuser/group/textvar[@name='StepStart']")]
        # converts date format of StepStart
        StepStart_iso = [datetime.datetime.strptime(StepStart[0].attrib['value'].split()[0], "%d/%m/%Y").strftime("%Y-%m-%d"),
                         datetime.datetime.strptime(StepStart[1].attrib['value'].split()[0], "%d/%m/%Y").strftime("%Y-%m-%d")]
        # creates date picker widgets with date set to XML input
        StepStart_picker = [ipywidgets.DatePicker(description       = 'Pre Start Date',
                                                  value             = datetime.date.fromisoformat(StepStart_iso[0]),
                                                  layout            = ipywidgets.Layout(width='40%'),
                                                  style             = {'description_width': '25ex'}),
                            ipywidgets.DatePicker(description       = 'Start Date',
                                                  value             = datetime.date.fromisoformat(StepStart_iso[1]),
                                                  layout            = ipywidgets.Layout(width='40%'),
                                                  style             = {'description_width': '25ex'})]
        # finds simulation end date in XML files
        StepEnd = [root[0].find("./lfuser/group/textvar[@name='StepEnd']"),
                   root[1].find("./lfuser/group/textvar[@name='StepEnd']")]
        # converts date format of StepStart
        StepEnd_iso = [datetime.datetime.strptime(StepEnd[0].attrib['value'].split()[0], "%d/%m/%Y").strftime("%Y-%m-%d"),
                       datetime.datetime.strptime(StepEnd[1].attrib['value'].split()[0], "%d/%m/%Y").strftime("%Y-%m-%d")]
        # creates date picker widgets with date set to XML input
        StepEnd_picker = [ipywidgets.DatePicker(description        = 'Pre Stop Date',
                                               value               = datetime.date.fromisoformat(StepEnd_iso[0]),
                                               layout              = ipywidgets.Layout(width='40%'),
                                               style               = {'description_width': '25ex'}),
                          ipywidgets.DatePicker(description        = 'Stop Date',
                                               value               = datetime.date.fromisoformat(StepEnd_iso[1]),
                                               layout              = ipywidgets.Layout(width='40%'),
                                               style               = {'description_width': '25ex'})]

        # creates empty list for slider widgets
        calibration_sliders = {}
        # finds calibration parameter group in XML files
        calibration_parameters_xml = [root[0].find("./lfuser")[2], root[1].find("./lfuser")[2]]
        # loops through calibration parameters listed in RUN settings file
        for element in calibration_parameters_xml[1]:
            # checks element type in XML file and allowed data range in parameter_specs dictionary
            if (element.tag == 'textvar' and element.attrib['name'] in parameter_specs):
                # creates slider widgets and appends to slider list
                calibration_sliders[element.attrib['name']] = ipywidgets.FloatSlider(
                                          value             = element.attrib['value'],
                                          min               = parameter_specs[element.attrib['name']][0],
                                          max               = parameter_specs[element.attrib['name']][1],
                                          step              = parameter_specs[element.attrib['name']][2],
                                          description       = element.attrib['name'],
                                          disabled          = False,
                                          continuous_update = False,
                                          orientation       = 'horizontal',
                                          readout           = True,
                                          readout_format    = parameter_specs[element.attrib['name']][3],
                                          layout            = ipywidgets.Layout(width='80%'),
                                          style             = {'description_width': '25ex'})

        # creates empty list for checkbox widgets
        module_checkboxes = {}
        # finds optional modules group in XML files
        optional_modules_xml = [root[0].find("./lfoptions"), root[1].find("./lfoptions")]
        # loops through optional modules listed in RUN settings file
        for element in optional_modules_xml[1]:
            # creates checkbox widgets and appends to checkbox list
            module_checkboxes[element.attrib['name']] = ipywidgets.Checkbox(
                                          value       = bool(int(element.attrib['choice'])),
                                          description = element.attrib['name'],
                                          disabled    = False,
                                          style       = {'description_width': '0ex'})

            
        # reads coordinates from XML RUN file
        coordinates = root[1].findall("./lfuser/group/textvar/[@name='Gauges']")[0].attrib['value']
        lon, lat = coordinates.split()
        center = (lat, lon)
        # creates interactive map with draggable marker
        m = ipyleaflet.Map(zoom=10, center=center, scroll_wheel_zoom=True)
        marker = ipyleaflet.Marker(location=center, draggable=True)
        m.add_layer(marker)
        # initializes visibility of map
        if module_checkboxes['repDischargeTs'].value == True:
            m.layout.display = "block"
        else:
            m.layout.display = "none"
            
        # creates tab widget
        tabs = ipywidgets.Tab()
        # sets tab titles
        tabs.set_title(0, 'Outputs')
        tabs.set_title(1, 'Date Selection')
        tabs.set_title(2, 'Calibration Parameters')
        tabs.set_title(3, 'Optional Modules')
        
        
        output_grid = ipywidgets.GridspecLayout(13, 3, height='auto')
        output_grid[0:8, 0] = ipywidgets.VBox([ipywidgets.HTML(value = '<b>Surface Water Outputs:</b>'),
                                             module_checkboxes['repDischargeMaps'],
                                             module_checkboxes['repDischargeTs'],
                                             module_checkboxes['repsimulateReservoirs'],
                                             module_checkboxes['repsimulateLakes'],
                                             module_checkboxes['repSnowCoverMaps'],
                                             module_checkboxes['repSnowMeltMaps'],
                                             module_checkboxes['repSurfaceRunoffMaps']])
        output_grid[0:9, 1] = ipywidgets.VBox([ipywidgets.HTML(value = '<b>Soil Outputs:</b>'),
                                             module_checkboxes['repThetaMaps'],
                                             module_checkboxes['repThetaForestMaps'],
                                             module_checkboxes['repThetaIrrigationMaps']])
        output_grid[0:9, 2] = ipywidgets.VBox([ipywidgets.HTML(value = '<b>Groundwater Outputs:</b>'),
                                             module_checkboxes['repPFMaps'],
                                             module_checkboxes['repPFForestMaps'],
                                             module_checkboxes['repUZMaps'],
                                             module_checkboxes['repLZMaps']])
        output_grid[9:13, 0] = ipywidgets.VBox([ipywidgets.HTML(value = '<b>Water Use Outputs:</b>'),
                                             module_checkboxes['repTotalAbs'],
                                             module_checkboxes['repTotalWUse']])
        output_grid[9:13, 1] = ipywidgets.VBox([ipywidgets.HTML(value = '<b>State and End Maps:</b>'),
                                             module_checkboxes['repStateMaps'],
                                             module_checkboxes['repEndMaps']])
                        
        # adds content to tabs
        tabs.children = [ipywidgets.VBox([output_grid, m]),
                        ipywidgets.VBox([StepStart_picker[0], StepEnd_picker[0], StepStart_picker[1], StepEnd_picker[1]]),
                        ipywidgets.VBox(list(calibration_sliders.values())),
                        ipywidgets.VBox([ipywidgets.GridBox([module_checkboxes['InitLisflood'],
                                                             module_checkboxes['InitLisfloodwithoutSplit'],
                                                             module_checkboxes['gridSizeUserDefined'],
                                                             module_checkboxes['SplitRouting'],
                                                             module_checkboxes['inflow'],
                                                             module_checkboxes['simulateReservoirs'],
                                                             module_checkboxes['simulateLakes'],
                                                             module_checkboxes['openwaterevapo'],
                                                             module_checkboxes['varfractionwater'],
                                                             module_checkboxes['drainedIrrigation'],
                                                             module_checkboxes['riceIrrigation'],
                                                             module_checkboxes['wateruse'],
                                                             module_checkboxes['useWaterDemandAveYear'],
                                                             module_checkboxes['TransientWaterDemandChange'],
                                                             module_checkboxes['wateruseRegion'],
                                                             module_checkboxes['groundwaterSmooth'],
                                                             module_checkboxes['indicator'],
                                                             module_checkboxes['readNetcdfStack'],
                                                             module_checkboxes['writeNetcdf'],
                                                             module_checkboxes['writeNetcdfStack']],
                                                             layout = ipywidgets.Layout(grid_template_columns = "33% 33% 33%"))])]
        
        if module_checkboxes['SplitRouting'].value == True:
            module_checkboxes['InitLisflood'].value = True
            module_checkboxes['InitLisfloodwithoutSplit'].value = False
            module_checkboxes['InitLisfloodwithoutSplit'].disabled = True
        else:
            module_checkboxes['InitLisflood'].value = False
            module_checkboxes['InitLisflood'].disabled = True
            module_checkboxes['InitLisfloodwithoutSplit'].value = True
                   
        module_checkboxes['SplitRouting'].observe(on_SplitRouting_clicked, names='value')
        module_checkboxes['repDischargeTs'].observe(on_repDischargeTs_clicked, names='value')

        # display options
        display(tabs)
        
        # button to start processing method
        processingButton = ipywidgets.Button(description="Start processing")
        processingButton.on_click(on_processingButton_clicked)
        display(processingButton)

#
def on_processingButton_clicked(b):
    # checks whether variable 'datasets' with reference to NetCDF files exists
    # if yes, closes all NetCDF files to re-write them
    # if no, creates empty list 'datasets'
    if 'datasets' in locals():
        for item in datasets:
            item[1].close()
        datasets.clear()
    else:
        datasets = []

    # loops through references to PRE-RUN and RUN settings files 
    for calibration_list in calibration_parameters_xml:
        # loops through calibration parameters listed in RUN settings file
        for element in calibration_list:
            # checks element type in XML file and allowed data range in parameter_specs dictionary
            if (element.tag == 'textvar' and element.attrib['name'] in parameter_specs):
                # updates value based on user input section
                #print('\nVorher ', element.attrib['value'], '\nNachher', str(calibration_sliders[element.attrib['name']].value), element.attrib)
                element.attrib['value'] = str(calibration_sliders[element.attrib['name']].value)


    # loops through references to PRE-RUN and RUN settings files 
    for module_list in optional_modules_xml:
        # loops through calibration parameters listed in RUN settings file
        for element in module_list:
            # updates value based on user input section
            #print('\nVorher ', element.attrib['choice'], '\nNachher', str(int(module_checkboxes[element.attrib['name']].value)), element.attrib)
            element.attrib['choice'] = str(int(module_checkboxes[element.attrib['name']].value))

    if module_checkboxes['SplitRouting'].value:
        optional_modules_xml[0].findall("setoption[@name='SplitRouting']")[0].attrib['choice'] = '1'
        optional_modules_xml[0].findall("setoption[@name='InitLisflood']")[0].attrib['choice'] = '1'
        optional_modules_xml[0].findall("setoption[@name='InitLisfloodwithoutSplit']")[0].attrib['choice'] = '0'
        optional_modules_xml[1].findall("setoption[@name='SplitRouting']")[0].attrib['choice'] = '1'
        optional_modules_xml[1].findall("setoption[@name='InitLisflood']")[0].attrib['choice'] = '0'
        optional_modules_xml[1].findall("setoption[@name='InitLisfloodwithoutSplit']")[0].attrib['choice'] = '0'
    else:
        optional_modules_xml[0].findall("setoption[@name='SplitRouting']")[0].attrib['choice'] = '0'
        optional_modules_xml[0].findall("setoption[@name='InitLisflood']")[0].attrib['choice'] = '0'
        optional_modules_xml[0].findall("setoption[@name='InitLisfloodwithoutSplit']")[0].attrib['choice'] = '1'
        optional_modules_xml[1].findall("setoption[@name='SplitRouting']")[0].attrib['choice'] = '0'
        optional_modules_xml[1].findall("setoption[@name='InitLisflood']")[0].attrib['choice'] = '0'
        optional_modules_xml[1].findall("setoption[@name='InitLisfloodwithoutSplit']")[0].attrib['choice'] = '0'

    # writes simulation start and stop date from user input section to both settings files
    for i in range(len(tree)):
        StepStart[i].attrib['value'] = '%s %s' % (str(datetime.datetime.strptime(str(StepStart_picker[i].value), "%Y-%m-%d").strftime("%d/%m/%Y")), StepStart[i].attrib['value'].split()[1])
        StepEnd[i].attrib['value']   = '%s %s' % (str(datetime.datetime.strptime(str(StepEnd_picker[i].value), "%Y-%m-%d").strftime("%d/%m/%Y")), StepEnd[i].attrib['value'].split()[1])
        tree[i].write(fc[i].selected)

    # starts LISFLOOD PRE-RUN and RUN according to (new) input
    path1 = fc[0].selected
    path2 = fc[1].selected

    print('PRE-RUN\n')
    !lisflood $path1
    print('\nRUN\n')
    !lisflood $path2
    print("\nDone")
       
# callback function to change map visibility
def on_repDischargeTs_clicked(button):
    if button['new']:
        m.layout.display = "block"
    else:
        m.layout.display = "none"

# 
def on_SplitRouting_clicked(button):
    if button['new']:
            module_checkboxes['InitLisflood'].disabled = False
            module_checkboxes['InitLisfloodwithoutSplit'].value = False
            module_checkboxes['InitLisfloodwithoutSplit'].disabled = True
    else:
            module_checkboxes['InitLisflood'].value = False
            module_checkboxes['InitLisflood'].disabled = True
            module_checkboxes['InitLisfloodwithoutSplit'].disabled = False        




####################################################
####################### CODE #######################
####################################################

# dictionary which contains editable calibration parameters including allowed data range
parameter_specs = {
    # parameter name           min   max    step  format
    "UpperZoneTimeConstant":  [0.01, 40,    0.01, '.2f'],
    "LowerZoneTimeConstant":  [1000, 10500, 5,    '.0f'],
    "GwPercValue":            [0.01, 2,     0.01, '.2f'],
    "GwLoss":                 [0,    0.5,   0.01, '.2f'],
    "b_Xinanjiang":           [0.5,  5,     0.01, '.2f'],
    "PowerPrefFlow":          [0.5,  8,     0.01, '.2f'],
    "CalChanMan":             [0.1,  20,    0.01, '.2f'],
    "SnowMeltCoef":           [2.5,  6.5,   0.01, '.2f'],
    "CalChanMan2":            [0.1,  20,    0.01, '.2f'],
    #"LZThreshold":            [0,    30,    0.01, '.2f'],
}

# creates list of file choosers for PRE-RUN and RUN settings file
fc = [FileChooser('/home/jovyan'), FileChooser('/home/jovyan')]
# sets title of file chooser
fc[0].title = '<b>Please select PRE-RUN Settings File:</b>'
fc[1].title = '<b>Please select RUN Settings File:</b>'
# initializes file chooser
for chooser in fc:
    # only show XML files and folders
    chooser.filter_pattern = '*.xml'
    # uses icon for folder to easier distinguish between folders and files
    chooser.use_dir_icons = True
    # callback function for RUN XML file
    chooser.register_callback(show_settings)
# display file chooser
ipywidgets.VBox([fc[0], fc[1]])

## Outputs

In [ ]:
# Click to EXPAND

####################################################
####################### CODE #######################
####################################################

# spatial outputs
# 
def subfinder(mylist, parameter):
    for i in range(len(mylist)):
        if mylist[i][0] == parameter:
            match = mylist[i][1]
    return match

# 
def update_time(date):
    mm.set_array(subfinder(datasets, parameter_dropdown.value)[parameter_dropdown.value][:][date].ravel())
    string_title=u'%s: %s' % (parameter_dropdown.value, date)
    plt.title(string_title, size='xx-large')

#  
def update_parameter(parameter):
    mm.set_array(subfinder(datasets, parameter)[parameter][:][date_slider.value].ravel())
    mm.autoscale()
    cbar.update_normal(mm)
    string_title=u'%s: %s' % (parameter, date_slider.value)
    plt.title(string_title, size='xx-large')

# check for spatial output files in output folder
if len(glob.glob('%s/out/*.nc' % fc[1].selected_path)) != 0:
    # list of paths to  all spatial output files
    paths_datasets = glob.glob('%s/out/*.nc' % fc[1].selected_path)
    datasets.clear()
    
    i = 0
    # open netCDF datasets
    for dataset in paths_datasets:
        datasets.append([dataset.split('/')[-1].split('.')[0], netCDF4.Dataset(paths_datasets[i])])
        i = i + 1

    # create drowdown menu with all available outputs
    parameter_dropdown = ipywidgets.Dropdown(options=[item[0] for item in datasets], description='Parameter:')
    # create date slider for given time period
    date_slider = ipywidgets.IntSlider(min=0, max=len(datasets[0][1].dimensions['time'])-1, step=1, value=0, description='Date:')
    # create simulation controls
    play = ipywidgets.Play(
            min = 0,
            max = len(datasets[0][1].dimensions['time'])-1,
            step = 1,
            description="Press play",
            disabled=False
    )

    # get chosen parameter
    parameter = parameter_dropdown.value

    # create figure with open street map
    request = cimgt.OSM()
    fig, ax = plt.subplots(subplot_kw=dict(projection=request.crs))
    ax.add_image(request, 8)
    cmap = plt.cm.cool
    vmin = np.min(subfinder(datasets, parameter)[parameter][:])
    vmax = np.percentile(subfinder(datasets, parameter)[parameter][:], 97.5)
    global mm
    mm = ax.pcolormesh(subfinder(datasets, parameter)['lon'][:],\
                                   subfinder(datasets, parameter)['lat'][:],\
                                   subfinder(datasets, parameter)[parameter][:][0],\
                                   vmin = vmin,\
                                   vmax = vmax, \
                                   transform = ccrs.PlateCarree(),\
                                   cmap = cmap,
                                   alpha = 0.5)
    global cbar
    cbar = plt.colorbar(mm, shrink = 0.7)
    cbar.ax.tick_params(labelsize=17)

    # update plot when user changes parameter in dropdown menu
    ipywidgets.interact(update_parameter, parameter = parameter_dropdown);
    # update plot when user changes date slider
    ipywidgets.interact(update_time, date = date_slider);
    # update plot when user starts simulation
    ipywidgets.jslink((play, 'value'), (date_slider, 'value'))

    display(play)
    
    
if len(glob.glob('%s/out/*.tss' % fc[1].selected_path)) != 0:
    pass

    def on_resetButton_clicked(b):
        df.drop(df.index, inplace=True)
        print('Reset successful.')





    if False:#len(df.index) != 0:

        selection = alt.selection_multi(fields=['setting'], bind='legend')
        chart = alt.Chart(df).mark_line(point=True).encode(
                            x       = 'date:T',
                            y       = 'value:Q',
                            color   = alt.Color('setting', legend=alt.Legend(title="Setting")),
                            opacity = alt.condition(selection, alt.value(1), alt.value(0.2))
                        ).add_selection(selection).interactive(bind_y=False)
        display(chart)
        display(run_settings.set_index('run'))

        resetButton = ipywidgets.Button(
            description  = 'Reset',
            disabled     = False,
            button_style = '', # 'success', 'info', 'warning', 'danger' or ''
            tooltip      = 'Reset',
            icon         = 'check' # (FontAwesome names without the `fa-` prefix)
        )
        resetButton.on_click(on_resetButton_clicked)
        display(resetButton)
    else:
        print('No data. Please run processing and try again.')



if len(glob.glob('%s/out/*.nc' % fc[1].selected_path)) == 0 and len(glob.glob('%s/out/*.tss' % fc[1].selected_path)) == 0:
    print('No output files in {}'.format(fc[1].selected_path))
